In [1]:
from scripts import *


Local stopwords:        True
GPUs detected:          1
Using GPU:              True
Device:                 cuda
Using cuML:             True
Got model from pickle:  True



In [2]:
userdf = pd.read_pickle(names["df-user"])

usergroupdf = pd.read_pickle(names["df-user-group"])

target_interest = pickle.load( open( names["target-interests"], "rb" ) )

print(f"""
  {userdf.shape}

  {usergroupdf.shape}

  {target_interest}
""")

usergroupdf.columns

first_day =  usergroupdf["time_day"].tail(1).values[0]
last_day = usergroupdf["time_day"].head(1).values[0]

print(f'''
first day: {first_day}
last day: {last_day}
ammount of days in df: {last_day - first_day}
''')


  (3803, 4)

  (89308, 6)

  ['add', 'fix', 'change', 'merge', 'update', 'test', 'remove', 'support', 'use', 'xla', 'build', 'request', 'pull', 'op', 'tests', 'tf', 'make', 'ops', 'error', 'api']


first day: 16746
last day: 19260
ammount of days in df: 2514



# Exploration

## Collaborative filtering

### Data Restructuring

- Grouping by user to get info on their commits and which target_interest their commits belong to in a quantitative way
- Using the user groups, we can again group the df by user groups and time and now have very few groups, and we can do regression on their activity over time

### User Grouping

In [3]:
#userdf_dict = userdf[["user","time_week","target_interest_word"]].copy()
userdf_with_dict = userdf[["user","target_interest"]].copy()


userdf_with_dict["target_interest_dict"] = userdf["target_interest"].apply(lambda x: dict_per_user(x, target_interest))

userdf_with_dict.drop(columns=["target_interest"], inplace=True)

print(len(userdf_with_dict))

userdf_with_dict.head(3)

3803


,user,target_interest_dict
0,(David) Siu-Kei Muk,"{'add': 0, 'fix': 1, 'change': 0, 'merge': 8, 'update': 0, 'test': 1, 'remove': 0, 'support': 0, 'use': 0, 'xla': 0, 'build': 0, 'request': 0, 'pull': 0, 'op': 0, 'tests': 0, 'tf': 0, 'make': 0, '..."
1,103yiran,"{'add': 0, 'fix': 0, 'change': 0, 'merge': 0, 'update': 0, 'test': 0, 'remove': 0, 'support': 0, 'use': 0, 'xla': 0, 'build': 0, 'request': 0, 'pull': 0, 'op': 0, 'tests': 0, 'tf': 0, 'make': 0, '..."
2,1e100,"{'add': 0, 'fix': 0, 'change': 0, 'merge': 0, 'update': 1, 'test': 0, 'remove': 3, 'support': 0, 'use': 0, 'xla': 0, 'build': 3, 'request': 0, 'pull': 0, 'op': 0, 'tests': 0, 'tf': 0, 'make': 0, '..."


In [4]:
# normalising the userdf
#userdf = (userdf-userdf.min())/(userdf.max()-userdf.min())

target_interest_matrix = np.array(userdf["target_interest"].apply(lambda x: dict_per_user(x, target_interest)))

df_user_interest_matrix = pd.DataFrame(list(target_interest_matrix))

target_interest_matrix = df_user_interest_matrix.to_numpy()

df_user_interest_matrix.insert(0, "user", userdf["user"])

df_user_interest_matrix

,user,add,fix,change,merge,update,test,remove,support,use,...,build,request,pull,op,tests,tf,make,ops,error,api
0,(David) Siu-Kei Muk,0,1,0,8,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,103yiran,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1e100,0,0,0,0,1,0,3,0,0,...,3,0,0,0,0,0,0,0,0,3
3,372046933,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4F2E4A2E,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3798,黄璞,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3799,黄鑫,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3800,박상준,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3801,이장후,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
target_interest_matrix[0:5]

array([[0, 1, 0, 8, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3],
       [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [6]:
colab_clusters = HDBSCAN(min_cluster_size=90, min_samples=10, metric='euclidean', cluster_selection_method='eom').fit(target_interest_matrix)

print(f"""
    Full dimensionality clustering output:
    Len of colab clusters: {len(colab_clusters.labels_)}
    Number of clusters: {len(set(colab_clusters.labels_)) - 1}
    Number of rows as outliers: {colab_clusters.labels_.tolist().count(-1)}
""")



    Full dimensionality clustering output:
    Len of colab clusters: 3803
    Number of clusters: 8
    Number of rows as outliers: 1374



In [7]:
colab_umap = UMAP(n_neighbors=15, min_dist=0.0).fit_transform(target_interest_matrix)

In [8]:
colab_resdf = pd.DataFrame({
    "x" : colab_umap[:, 0], 
    "y" : colab_umap[:, 1], 
    "cluster" : colab_clusters.labels_
})

# with few clusters you can turn on and off outliers with the -1 label
#colab_resdf = colab_resdf[colab_resdf["cluster"] != -1]

#turning cluster to str for discrete color
colab_resdf["cluster"] = colab_resdf["cluster"].astype(str)

fig_colab = px.scatter(colab_resdf, x="x", y="y", color="cluster", title="Colab clustering", width=800, height=800, range_x=[-25, 25], range_y=[-25, 25])

fig_colab.show()

# TODO add one visualisation without time grouping
# this would give us the "true" user groups, and then we could see if they moved around without breaking up the group too much
# also it is not a bug that there is overlap of clusters, as the clustering takes place before umap

In [9]:
## TODO
# - now we have to do this per user.
# - we need to look at what a given user is "comitting" about interest wise, and then see which cluster that user is in
# - then when we    title="Timeline of commits by interest",                                                         

### user cluster time grouping

In [10]:
# making dict to connect username and cluster id
userdId_groupID_dict = dict(zip(df_user_interest_matrix["user"], colab_clusters.labels_))

if len(userdId_groupID_dict) - len(df_user_interest_matrix) != 0:
    print("WARNING: dict and userdf_ex are not the same length")

### Missing data interpolation

- Insert nan rows per missing date
- Fill nan with smoothed values
  - (this is called interpolation)

In [11]:
days_series = pd.Series(range(first_day, last_day + 1))

In [12]:
usergroupdf = usergroupdf.groupby(["user_group_id", "time_day"]).agg(list).reset_index()


print(f'User group equal to cluster groups: {len(usergroupdf["user_group_id"].unique()) == len(set(colab_clusters.labels_))}')
print(len(usergroupdf))


usergroupdf.head(3)

User group equal to cluster groups: True
5400


,user_group_id,time_day,user,time_sec,target_interest_id,target_interest
0,-1,16746,"[Vijay Vasudevan, Vijay Vasudevan, Vijay Vasudevan, Vijay Vasudevan, Manjunath Kudlur, Manjunath Kudlur]","[1446935606, 1446933504, 1446922181, 1446875858, 1446863831, 1446856078]","[0, 16, 5, 0, 5, 0]","[api, tf, update, api, update, api]"
1,-1,16747,"[Manjunath Kudlur, Manjunath Kudlur, Vijay Vasudevan]","[1447024477, 1447019816, 1447011446]","[0, 5, 5]","[api, update, update]"
2,-1,16748,"[Manjunath Kudlur, Manjunath Kudlur, Manjunath Kudlur, Manjunath Kudlur, Manjunath Kudlur, Manjunath Kudlur, Manjunath Kudlur]","[1447049284, 1447047697, 1447046850, 1447045469, 1447039735, 1447039421, 1447033308]","[0, 0, 16, 0, 16, 0, 0]","[api, api, tf, api, tf, api, api]"


In [13]:
# making the interest matrix again for user groups
usergroup_target_interest_matrix = np.array(usergroupdf["target_interest"].apply(lambda x: dict_per_user(x, target_interest)))

df_group_interest_matrix = pd.DataFrame(list(usergroup_target_interest_matrix))

usergroup_target_interest_matrix = df_group_interest_matrix.to_numpy()

#df_usergroup_interest_matrix = usergroupdf[["user_group_id", "time_day", "target_interest"]].copy()

df_group_interest_matrix.insert(0, "user_group_id", usergroupdf["user_group_id"])
df_group_interest_matrix.insert(1, "time_day", usergroupdf["time_day"])


df_group_interest_matrix

,user_group_id,time_day,add,fix,change,merge,update,test,remove,support,...,build,request,pull,op,tests,tf,make,ops,error,api
0,-1,16746,0,0,0,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,3
1,-1,16747,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,-1,16748,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,5
3,-1,16749,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,-1,16750,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,7,19208,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5396,7,19221,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5397,7,19227,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
5398,7,19228,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [14]:
df_group_interest_matrix.describe()

,user_group_id,time_day,add,fix,change,merge,update,test,remove,support,...,build,request,pull,op,tests,tf,make,ops,error,api
count,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,...,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000
mean,0.864259,17945.749074,0.123519,1.177222,0.469815,1.148148,0.720370,1.065741,0.558148,0.044074,...,0.473148,0.023148,0.024630,0.022222,0.260741,1.633889,0.060556,0.173148,0.380000,7.182778
std,2.362950,696.042434,0.426614,1.986498,1.045110,2.410290,1.720504,2.144945,1.283181,0.233161,...,1.084681,0.171127,0.181433,0.153573,0.716653,2.914132,0.273277,0.561978,0.807476,11.033168
min,-1.000000,16746.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.000000,17347.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,17912.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,2.000000,18528.000000,0.000000,2.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000,11.000000
max,7.000000,19260.000000,8.000000,27.000000,10.000000,29.000000,50.000000,23.000000,28.000000,4.000000,...,14.000000,3.000000,3.000000,2.000000,10.000000,32.000000,3.000000,17.000000,10.000000,124.000000


In [15]:
days_frame = pd.DataFrame({"time_day" : days_series})

In [16]:
# making list of dfs based on usergroup

usergroup_df_list = []

for usergroup in df_group_interest_matrix["user_group_id"].unique():
  # interpolationg the missing days
  
  df_this_group = df_group_interest_matrix[df_group_interest_matrix["user_group_id"] == usergroup]

  df_this_group.drop(columns=["user_group_id"], inplace=True)

  df_this_group = pd.merge(days_frame, df_this_group, how="outer", on="time_day")

  #df_this_group.set_index("time_day", inplace=True)

  for columns in df_this_group.columns:
    if columns != "time_day":
      df_this_group[columns][0] = 0.0
      df_this_group[columns] = df_this_group[columns].astype(float)
      df_this_group[columns].interpolate(method="spline", order=3, inplace=True) # cubic, tried linear, but it was not as good
      df_this_group.fillna(method="ffill", inplace=True)

  #df_this_group = df_this_group.interpolate(method="pad", axis=0)
  
  usergroup_df_list.append(df_this_group)

In [17]:
for x in usergroup_df_list:
  print(x.shape)

(2515, 21)
(2515, 21)
(2515, 21)
(2515, 21)
(2515, 21)
(2515, 21)
(2515, 21)
(2515, 21)
(2515, 21)


In [18]:
usergroup_df_list[5].isna().sum()

time_day    0
add         0
fix         0
change      0
merge       0
update      0
test        0
remove      0
support     0
use         0
xla         0
build       0
request     0
pull        0
op          0
tests       0
tf          0
make        0
ops         0
error       0
api         0
dtype: int64

In [19]:
usergroup_df_list[5]

,time_day,add,fix,change,merge,update,test,remove,support,use,...,build,request,pull,op,tests,tf,make,ops,error,api
0,16746,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0
1,16747,0.003914,0.0,-0.001536,0.010850,-0.009369,0.095102,0.269866,0.0,-0.002036,...,0.048427,0.0,0.0,0.0,0.0,-0.006305,0.0,0.069391,0.79806,0.0
2,16748,0.003881,0.0,-0.001530,0.010802,-0.009257,0.094826,0.269602,0.0,-0.002022,...,0.048328,0.0,0.0,0.0,0.0,-0.006283,0.0,0.069195,0.79836,0.0
3,16749,0.003848,0.0,-0.001523,0.010754,-0.009144,0.094549,0.269339,0.0,-0.002008,...,0.048229,0.0,0.0,0.0,0.0,-0.006261,0.0,0.068999,0.79866,0.0
4,16750,0.003815,0.0,-0.001516,0.010705,-0.009032,0.094274,0.269075,0.0,-0.001994,...,0.048130,0.0,0.0,0.0,0.0,-0.006239,0.0,0.068803,0.79896,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,19256,-0.018092,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.00000,0.0
2511,19257,-0.018179,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.00000,0.0
2512,19258,-0.018266,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.00000,0.0
2513,19259,-0.018353,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.00000,0.0


### Neural Net Custom Model
This will be scuffed


In [46]:
from torch import nn
import copy
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [21]:
# we do not need flatten as we do not have 2D input

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        

        # nn.Sequential sets the layers in order
        self.linear_relu_stack = nn.Sequential(

          # our input is 20
            nn.Linear(20, 40),
            nn.ReLU(),

            # adding two more hidden layers
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),

          # our output is 20
            nn.Linear(40, 20),
            nn.ReLU(),
            #nn.Softmax(dim=1)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

### Training the Model

In [35]:
from math import floor

# taking one of the usergroup dfs to test
sampledf = usergroup_df_list[1]

print(len(sampledf))

group_tensor = torch.tensor(sampledf.drop(columns=["time_day"]).values, device=device).float()


start = floor(len(group_tensor) * (7/10))
end = len(group_tensor) - start

group_tensor_train, group_tensor_test = torch.split(group_tensor, [start, end])


print(f"""
{group_tensor.size()}
{group_tensor_train.size()}
{group_tensor_test.size()}
""")

len(group_tensor[0])

2515

torch.Size([2515, 20])
torch.Size([1760, 20])
torch.Size([755, 20])



,time_day,add,fix,change,merge,update,test,remove,support,use,...,build,request,pull,op,tests,tf,make,ops,error,api
0,16746,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,16747,0.003199,0.076081,-0.006453,0.057722,0.033281,0.025965,0.027483,0.002682,-0.002068,...,-0.021477,0.0,-0.009255,0.0,-0.004170,-0.043662,0.001833,0.002623,0.145204,1.130167
2,16748,0.003183,0.076185,-0.006421,0.057605,0.033310,0.025898,0.027460,0.002671,-0.002064,...,-0.021365,0.0,-0.009202,0.0,-0.004146,-0.043363,0.001823,0.002639,0.145005,1.130583
3,16749,0.003167,0.076288,-0.006389,0.057487,0.033339,0.025832,0.027438,0.002659,-0.002060,...,-0.021254,0.0,-0.009150,0.0,-0.004121,-0.043065,0.001813,0.002656,0.144807,1.130998
4,16750,0.003152,0.076391,-0.006356,0.057370,0.033368,0.025765,0.027415,0.002647,-0.002056,...,-0.021144,0.0,-0.009098,0.0,-0.004096,-0.042767,0.001803,0.002672,0.144609,1.131412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,19256,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
2511,19257,0.001388,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
2512,19258,0.001376,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
2513,19259,0.001364,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000


In [60]:
def train(tensor, model, loss_fn, optimizer):
   running_loss = 0.0

    # this is for the graph - we just append the first 2 days
   prediction_row_tensor = tensor[0:1]

   for i in range(len(tensor) - 1): # - 1 because we are getting the next row yeee
     this_row = tensor[i]
     next_row = tensor[i+1]

     next_row_prediction = model(this_row)
        
     # this is just for the graph - here we append the next row 
     prediction_row_tensor = torch.cat((prediction_row_tensor, next_row_prediction[None, ...]), dim=0)

     loss = loss_fn(next_row_prediction, next_row)

     # backpropagation
     optimizer.zero_grad()
     loss.backward()
     optimizer.step()

     running_loss += loss.item()
   return running_loss / len(tensor), prediction_row_tensor

def mutate(best_model, model, mutation_rate):
    for param, best_param in zip(model.parameters(), best_model.parameters()):
      #if param.requires_grad:
      param.data = best_param.data + mutation_rate * torch.randn_like(best_param)

In [61]:
# this is the loss function we want to use? MSE is often used when predicting numerical stuff
criterion = nn.MSELoss()

model_count = 3
epochs = 2
generations = 10
mutation_rate = 0.2

# creating multiple networks to train
models = [NeuralNetwork().to(device) for _ in range(model_count)]
optimizers = [torch.optim.SGD(model.parameters(), lr=1e-3) for model in models]
losses = [0.0 for _ in range(model_count)]

# NEAT
for gen in range(generations):
  for i in range(model_count):
    for epoch in range(epochs):
      loss, prediction_row_tensor = train(group_tensor_train, models[i], criterion, optimizers[i])
      losses[i] = loss
      print(f"generation {gen} epoch {epoch}, model {i}, loss: {loss}")  
  
  # we want to keep the best model
  best_model_index = losses.index(min(losses))
  print(f"generation {gen}, best model {best_model_index}")
  for m in range(model_count):
    if m != best_model_index:
      mutate(models[best_model_index], models[m], mutation_rate)

generation 0 epoch 0, model 0, loss: 0.11949483467350629
generation 0 epoch 1, model 0, loss: 0.1186237874102186
generation 0 epoch 0, model 1, loss: 0.12056842418387533
generation 0 epoch 1, model 1, loss: 0.11907228036275641
generation 0 epoch 0, model 2, loss: 0.11558059230023487
generation 0 epoch 1, model 2, loss: 0.09452187993288548
generation 0, best model 2
generation 1 epoch 0, model 0, loss: 0.1556731364976837
generation 1 epoch 1, model 0, loss: 0.05504936269714147
generation 1 epoch 0, model 1, loss: 0.13371866708121988
generation 1 epoch 1, model 1, loss: 0.05074352048864532
generation 1 epoch 0, model 2, loss: 0.07966809833835048
generation 1 epoch 1, model 2, loss: 0.07037835638276936
generation 1, best model 1
generation 2 epoch 0, model 0, loss: 0.30972945696834026
generation 2 epoch 1, model 0, loss: 0.08979243563457066
generation 2 epoch 0, model 1, loss: 0.04513083242901659
generation 2 epoch 1, model 1, loss: 0.04294199696006464
generation 2 epoch 0, model 2, loss:

In [62]:
prediction_row_tensor.size()

torch.Size([1760, 20])

In [63]:
group_tensor[5]

tensor([ 0.0031,  0.0765, -0.0063,  0.0573,  0.0334,  0.0257,  0.0274,  0.0026,
        -0.0021, -0.0116, -0.0210,  0.0000, -0.0090,  0.0000, -0.0041, -0.0425,
         0.0018,  0.0027,  0.1444,  1.1318], device='cuda:0')

### Predicting / Looking at results

In [64]:
# getting the predictions for the test set

for row in group_tensor_test:
    prediction_row_tensor = torch.cat((prediction_row_tensor, model(row)[None, ...]), dim=0)

In [65]:
start

1760

In [66]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [67]:
prl = prediction_row_tensor.cpu().detach().numpy()
gp = group_tensor.cpu().detach().numpy()

fig = px.line(pd.DataFrame(list(prl))[start: ], height=600, width=1200, title="Predictions")# [start: ]

fig.show()
fig.write_html(f"figures/ml-predictions{timestamp}.html")

In [68]:
fig = px.line(pd.DataFrame(list(gp))[start: ], height=600, width=1200, title="Real") # [start: ]

fig.show()
fig.write_html(f"figures/ml-baseline{timestamp}.html")

In [69]:
print(f"day {start} is equal to week {start//7}")

for i, interest in enumerate(target_interest):
  print(f"value {i:2g} => {interest}")

day 1760 is equal to week 251
value  0 => add
value  1 => fix
value  2 => change
value  3 => merge
value  4 => update
value  5 => test
value  6 => remove
value  7 => support
value  8 => use
value  9 => xla
value 10 => build
value 11 => request
value 12 => pull
value 13 => op
value 14 => tests
value 15 => tf
value 16 => make
value 17 => ops
value 18 => error
value 19 => api


## Exploring water simulation based prediction potential

In [70]:

print(f"""
    Bounds of the uembs
    
    x axis:
    {min(uembs[:,0])}
    {max(uembs[:,0])}
    
    "y axis"
    {min(uembs[:,1])}
    {max(uembs[:,1])}
""")

NameError: name 'uembs' is not defined

Kan sette ramme til vann prediction på +- 25 på begge akser

512 x 512*2 pixels i det spacet

lage neste frame i animasjonen

gi to frames av fortid
- kan gi en frame per uke per bruker
- kan ha en farge per bruker gruppe